In [148]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import to_timestamp, col, month, dayofmonth, desc

In [116]:
# Create a sapr
spark = SparkSession.builder \
    .appName("SparkSQLExampleAPP") \
    .getOrCreate()

# Set log level to ERROR to reduce verbosity
spark.sparkContext.setLogLevel("ERROR")

In [123]:
# create schema
schema = "`date` string, `delay` INT, `distance` INT, `origin` STRING, `destination` STRING" 


In [124]:
df = spark.read.csv('departuredelays.csv', schema=schema, header=True)

In [125]:
df.printSchema()

root
 |-- date: string (nullable = true)
 |-- delay: integer (nullable = true)
 |-- distance: integer (nullable = true)
 |-- origin: string (nullable = true)
 |-- destination: string (nullable = true)



In [126]:
df.show(5)

+--------+-----+--------+------+-----------+
|    date|delay|distance|origin|destination|
+--------+-----+--------+------+-----------+
|01011245|    6|     602|   ABE|        ATL|
|01020600|   -8|     369|   ABE|        DTW|
|01021245|   -2|     602|   ABE|        ATL|
|01020605|   -4|     602|   ABE|        ATL|
|01031245|   -4|     602|   ABE|        ATL|
+--------+-----+--------+------+-----------+
only showing top 5 rows



In [137]:
df = df.withColumn('flight_date', to_timestamp(col('date'), 'MMddHHmm'))

In [138]:
df.show(5)

+--------+-----+--------+------+-----------+-------------------+
|    date|delay|distance|origin|destination|        flight_date|
+--------+-----+--------+------+-----------+-------------------+
|01011245|    6|     602|   ABE|        ATL|1970-01-01 12:45:00|
|01020600|   -8|     369|   ABE|        DTW|1970-01-02 06:00:00|
|01021245|   -2|     602|   ABE|        ATL|1970-01-02 12:45:00|
|01020605|   -4|     602|   ABE|        ATL|1970-01-02 06:05:00|
|01031245|   -4|     602|   ABE|        ATL|1970-01-03 12:45:00|
+--------+-----+--------+------+-----------+-------------------+
only showing top 5 rows



In [139]:
## Create a temporary view - this only exists for the duration of the Spark session
df.createOrReplaceTempView('us_delay_flights_tbl')

In [140]:
spark.sql(
    """
select *
from us_delay_flights_tbl
limit 5
"""
).show()

+--------+-----+--------+------+-----------+-------------------+
|    date|delay|distance|origin|destination|        flight_date|
+--------+-----+--------+------+-----------+-------------------+
|01011245|    6|     602|   ABE|        ATL|1970-01-01 12:45:00|
|01020600|   -8|     369|   ABE|        DTW|1970-01-02 06:00:00|
|01021245|   -2|     602|   ABE|        ATL|1970-01-02 12:45:00|
|01020605|   -4|     602|   ABE|        ATL|1970-01-02 06:05:00|
|01031245|   -4|     602|   ABE|        ATL|1970-01-03 12:45:00|
+--------+-----+--------+------+-----------+-------------------+



In [141]:
spark.sql("""select date, delay, distance, origin, destination 
          from us_delay_flights_tbl
          where distance > 1000
          order by distance desc
          """).show()

+--------+-----+--------+------+-----------+
|    date|delay|distance|origin|destination|
+--------+-----+--------+------+-----------+
|03011625|   -1|    4330|   HNL|        JFK|
|01011625|   -4|    4330|   HNL|        JFK|
|02011625|   -1|    4330|   HNL|        JFK|
|01211625|  115|    4330|   HNL|        JFK|
|03291530|   37|    4330|   HNL|        JFK|
|01021625|  110|    4330|   HNL|        JFK|
|02271625|   -7|    4330|   HNL|        JFK|
|01031625|   -1|    4330|   HNL|        JFK|
|03021625|   14|    4330|   HNL|        JFK|
|01041625|   -7|    4330|   HNL|        JFK|
|02021625|   -5|    4330|   HNL|        JFK|
|01051625|   18|    4330|   HNL|        JFK|
|03051625|   -6|    4330|   HNL|        JFK|
|01061625|    0|    4330|   HNL|        JFK|
|02051625|   -8|    4330|   HNL|        JFK|
|01071625|    0|    4330|   HNL|        JFK|
|03061625|   -2|    4330|   HNL|        JFK|
|01081625|   -5|    4330|   HNL|        JFK|
|02061625|   -9|    4330|   HNL|        JFK|
|01091625|

In [142]:
spark.sql("""select date, delay, distance, origin, destination
          from fire_calls
          where origin = 'SFO' and destination = 'ORD'
                and delay >= 120
          order by delay desc
          """).show()

+--------+-----+--------+------+-----------+
|    date|delay|distance|origin|destination|
+--------+-----+--------+------+-----------+
|02190925| 1638|    1604|   SFO|        ORD|
|01031755|  396|    1604|   SFO|        ORD|
|01022330|  326|    1604|   SFO|        ORD|
|01051205|  320|    1604|   SFO|        ORD|
|01190925|  297|    1604|   SFO|        ORD|
|02171115|  296|    1604|   SFO|        ORD|
|01071040|  279|    1604|   SFO|        ORD|
|01051550|  274|    1604|   SFO|        ORD|
|03120730|  266|    1604|   SFO|        ORD|
|01261104|  258|    1604|   SFO|        ORD|
|01161210|  225|    1604|   SFO|        ORD|
|02091800|  223|    1604|   SFO|        ORD|
|01221040|  215|    1604|   SFO|        ORD|
|03121155|  203|    1604|   SFO|        ORD|
|02111256|  197|    1604|   SFO|        ORD|
|03311405|  196|    1604|   SFO|        ORD|
|01031920|  193|    1604|   SFO|        ORD|
|01021410|  190|    1604|   SFO|        ORD|
|03171215|  189|    1604|   SFO|        ORD|
|01101410|

In [147]:
spark.sql("""
select delay, origin, destination,
          case
            when delay > 360 then 'Very Long Delays'
            when delay >= 120 and delay <= 120 then 'Short Delays'
            when delay >= 60 and delay < 120 then 'Short Delays'
            when delay > 0 and delay <60 then 'Tolerable Delays'
            when delay = 0 then 'No Delays'
            else 'Early'
            end as flight_delays
          from us_delay_flights_tbl
          order by delay desc         
""").show(10)

+-----+------+-----------+----------------+
|delay|origin|destination|   flight_delays|
+-----+------+-----------+----------------+
| 1642|   TPA|        DFW|Very Long Delays|
| 1638|   SFO|        ORD|Very Long Delays|
| 1636|   FLL|        DFW|Very Long Delays|
| 1592|   RSW|        ORD|Very Long Delays|
| 1560|   BNA|        DFW|Very Long Delays|
| 1553|   PDX|        DFW|Very Long Delays|
| 1543|   CLE|        DFW|Very Long Delays|
| 1511|   MCO|        ORD|Very Long Delays|
| 1500|   EGE|        JFK|Very Long Delays|
| 1496|   ONT|        DFW|Very Long Delays|
+-----+------+-----------+----------------+
only showing top 10 rows



In [153]:
(df.select('distance', 'origin', 'destination') \
    .where(col('distance') > 1000) \
    .orderBy(desc('distance'))).show(10)

+--------+------+-----------+
|distance|origin|destination|
+--------+------+-----------+
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
+--------+------+-----------+
only showing top 10 rows



In [156]:
df.select('distance','origin', 'destination') \
    .where('distance > 1000') \
    .orderBy('distance', ascending = False).show(10)

+--------+------+-----------+
|distance|origin|destination|
+--------+------+-----------+
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
+--------+------+-----------+
only showing top 10 rows

